In [1]:
import os, sys

sys.path.append(os.path.abspath(os.path.join('..')))

from dotenv import load_dotenv

load_dotenv(dotenv_path="../.env.local")

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {os.getenv('TMDB_API_KEY')}",
}

In [2]:
from movie_utils import *

from utils import *

c:\Me\Duke\Classes\Summer '24\MLOps\cinelens\api\.venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [3]:
# titles = [
#     'Captain America: Civil War',
#     'Abigail',
#     'Mission: Impossible - Fallout',
#     'John Wick: Chapter 4',
#     'Se7en'
#     ]

# movies = [
#     get_details(title, headers)
#     for title in titles
# ]

# movies_new = []

# for movie in movies:
#     movies_new.append(
#         {
#             'title': movie['title'],
#             'overview': movie['overview'],
#             'release_date': movie['release_date'],
#             'rating': round(movie['rating'] / 2, 1),
#             'tagline': movie['tagline'],
#             'poster': movie['poster'],
#             'backdrop': movie['backdrop'],
#             'runtime': movie['runtime'],
#             'genres': (movie['genres']),
#             'providers': get_new_providers(movie['providers'])
#         }
#     )

In [4]:
client = connect_to_mongo()

query = """
A successful banker is wrongfully convicted of murdering his wife and her lover and is sentenced to life
imprisonment in a harsh penitentiary. Over the decades, he forms an unlikely friendship with a fellow inmate,
a seasoned contraband smuggler. Despite the brutality and corruption within the prison walls, he maintains
hope and dignity, using his financial skills to assist the prison staff, ultimately orchestrating a daring
and ingenious escape plan. The narrative explores themes of hope, resilience, and the enduring human spirit in the face of adversity.
"""

# query = """
# A movie with Batman, Jim Gordon, and Harvey Dent. 
# It starts Christian Bale and Michael Caine.
# It's directed by Christopher Nolan.
# """

resp = query_rag_movie(text=query, client=client)

client.close()


Pinged your deployment. You successfully connected to MongoDB!


In [5]:
print(resp)

["The Shawshank Redemption", "Escape from Alcatraz", "The Green Mile", "Papillon", "Cool Hand Luke"]


In [6]:
eval(resp)

['The Shawshank Redemption',
 'Escape from Alcatraz',
 'The Green Mile',
 'Papillon',
 'Cool Hand Luke']